# 1.데이터 준비와 전처리

In [6]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [10]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [11]:
print(ratings_cols)

['user_id', 'movie_id', 'rating', 'timestamp']


In [58]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# 2.분석해 봅시다.

In [22]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [34]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [28]:
#가장 인기있는 영화 30개(인기순)
top_30_movie_count = ratings.groupby('movie_id')['user_id'].count()
top_30_movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

# 3.내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [42]:
ratings

my_ratings = pd.DataFrame({'user_id': ['6041']*5, 'movie_id' : [17, 25, 35, 7, 46] , 'count':[4, 5, 4, 3, 5], 'timestamp' : [956715570, 956715571, 956715572, 956715573, 956715574],})
new_ratings = ratings.append(my_ratings)
new_ratings.tail(10)

new_ratings.shape, ratings.shape

((836483, 4), (836478, 4))

# 4.  CSR matrix를 직접 만들어 봅시다.

In [65]:
user_idx_to_unique = {v: k for k, v in enumerate(new_ratings.user_id.unique())}
movie_idx_to_unique = {v: k for k, v in enumerate(new_ratings.movie_id.unique())}
# 누락된 값 보정

In [84]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = new_ratings[new_ratings['movie_id']<2]
one, all_new_ratings = len(only_one), len(new_ratings)
print(f'{one},{all_new_ratings}')
print(f'Ratio of only_one over all data is {one/all_new_ratings:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

2000,836483
Ratio of only_one over all data is 0.24%


In [94]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = new_ratings['user_id'].nunique()
num_movie_id = new_ratings['movie_id'].nunique()

csr_data = csr_matrix((new_ratings.count, (new_ratings.user_id, new_ratings.movie_id)))
csr_data

TypeError: '>=' not supported between instances of 'int' and 'str'